In [ ]:
# 一、选股,股池：
# (聚宽上我还没实现股池管理)
# 1、每日全市场选股入股池:
# (1)剔除st,退市、前一交易日停牌的股
# (2)选股条件：
# 连续三日缩量，当日量小于五日十日十五日均量
# (3)若标的已在股池中，选股日期设置为最新日期
# (4)每日清除在股池中超过5个交易日的标的
# 二、买入：
# 1、股池中符合买入条件:
# (1)当日量大于前一日量
# (2)当日量大于五日十日十五日均量
# (3)当日收盘价大于开盘价
# (4)当日KDJ金叉
# (5)买入前以换手及五日平均换手排序，换手大者优先买入(聚宽上我还没实现)
# (6)大盘23个交易日内出现macd死叉不买(刚想到，聚宽上我还没实现)


In [ ]:
import pandas as pd
import numpy as np

In [ ]:

def FILTERx(COND, N):

    k1 = pd.Series(np.where(COND, 1, 0), index=COND.index)
    idx = k1[k1 == 1].index.codes[0]
    needfilter = pd.Series(idx, index=idx)
    afterfilter = needfilter.diff().apply(lambda x: False if x > N else True)
    k1.iloc[afterfilter[afterfilter].index] = 2
    return k1.apply(lambda x: 1 if x == 2 else 0)

In [ ]:
def select1(data):
    # 连续三日缩量
    ch= data.close[-1] * 1.1
    cl= data.close[-1] * 0.9
#     ch= data.close * 1.1
#     cl = data.close * 0.9
    
    df=pd.concat([QA.MA(data.close, x) for x in (5,10,20,30,60,90,120,250) ], axis = 1).dropna()[-1:]
    df.columns = [u'm5',u'm10',u'm20',u'm30',u'm60',u'm90',u'm120', u'm250']  
    df_h = df.apply(lambda x:x.max() <= ch,  axis = 1 )
    df_l = df.apply(lambda x:x.min() >= cl,  axis = 1 )
    
    df['dfh'] = df_h
    df['dfl'] = df_l
#     out=df.iloc[-1].apply(lambda x: True if x>cl and x < ch else False)

    return df
 
#     return pd.DataFrame({
# #         't1': t1,
# #         't2': t2,
# #         't3': t3,
# #         'm5': MA_N[0],
# #         'm10':  MA_N[1],
# #         'm20':  MA_N[2],
# #         'm30':  MA_N[3],
# #         'm60':  MA_N[4],
# #         'm90':  MA_N[5],
# #         'm120':  MA_N[6],
# #         'm250':  MA_N[7],
#         'out':out,
# #         'man':MA_N
#     })
    


In [ ]:
#QA.QA_fetch_stock_block_adv().get_block('上证50').code

In [ ]:
import QUANTAXIS as QA

In [ ]:
# stock_list = QA.QA_fetch_stock_block_adv().get_block('中小300').code

In [32]:
stock_list = QA.QA_fetch_stock_list_adv()

In [42]:
stocklist_all = stock_list[~stock_list.name.apply(lambda x: 'ALL-ST' in x)].code.tolist()

In [43]:
# stocklist_all=stocklist_all[1:5]

In [44]:
# stocklist_all

In [45]:
#blockname=QA.QA_fetch_stock_block_adv().block_name

In [46]:
data = QA.QA_fetch_stock_day_adv(stocklist_all,'2010-01-01','2020-11-30')

In [ ]:
data.selects('000002','2018-06-10').data

In [ ]:
# = QA.QA_fetch_stock_day_adv('000001','2019-07-01','2019-09-30')

In [ ]:
#data.selects('600532','2008-05-28','2008-06-10').data

In [47]:
ind = data.add_func(select1)
ind.query('dfh==True and dfl==True')

m5     m10      m20        m30        m60        m90  \
date       code                                                               
2020-04-07 000002  26.600  26.321  27.1160  28.316333  29.057667  29.175111   
           000010   3.516   3.514   3.5005   3.527667   3.544000   3.490556   
           000011   8.522   8.428   8.2315   8.186667   8.385500   8.664111   
           000012   4.480   4.544   4.7190   5.002667   4.907333   4.812333   
           000025  18.868  18.550  18.5250  18.962333  19.424000  19.811111   
           000027   5.410   5.412   5.4090   5.470000   5.636833   5.795333   
           000028  40.146  40.161  40.2565  41.754333  45.268333  44.831778   
2016-09-13 000029  10.994  11.036  11.2775  11.207333  10.879167  10.584333   
2020-04-07 000030   5.366   5.333   5.6185   5.786667   5.236500   5.047222   
           000037  11.466  11.561  11.6865  11.480000  11.389333  11.739778   
           000042   9.416   9.389   9.5135   9.620667   9.724500   9.960222   
           000045   7.068   7.036   7.4150   7.685667   7.621167   7.547111   
           000048  23.078  22.364  21.9585  22.352333  22.201667  22.024333   
           000059   5.678   5.633   5.7655   5.878000   5.845167   5.871333   
           000096  10.726  10.496  10.6720  10.745333  10.883667  10.912333   
           000153   6.918   6.852   6.7955   6.967000   7.053500   6.811333   
           000157   5.746   5.787   5.7545   6.000667   6.200000   6.228000   
           000301   4.828   4.809   4.8090   4.897667   4.964667   5.009333   
           000338  12.570  12.565  12.8375  13.481000  13.892833  14.053111   
           000407   3.614   3.574   3.5020   3.436667   3.431333   3.443667   
           000411  11.424  11.450  11.5725  11.943667  12.143667  12.056556   
           000417   4.738   4.762   4.7005   4.655000   4.738000   4.743556   
           000419   4.106   4.129   4.2920   4.270000   4.301333   4.343778   
           000420   2.188   2.207   2.4610   2.450333   2.296500   2.252556   
           000421   4.116   4.130   4.1860   4.218000   4.277667   4.333000   
           000428   2.858   2.949   3.0330   2.981333   2.718333   2.678333   
           000488   4.894   4.733   4.6825   4.717000   4.813000   4.805000   
           000501  11.170  11.231  11.1340  11.140667  11.481500  11.613444   
           000502   6.754   6.909   7.6155   7.484000   7.271667   7.214333   
           000510   3.946   3.989   4.0810   4.306333   4.277333   4.174556   
...                   ...     ...      ...        ...        ...        ...   
           603278   9.442   9.845   9.7630   9.269667   8.938333   8.930333   
           300721  19.028  19.785  20.4655  20.421333  20.377000  19.708111   
           002864  26.156  26.345  26.3245  26.745333  27.488833  27.046778   
           300723  41.714  41.663  40.4530  40.965333  40.890500  39.096889   
           300726  25.030  25.331  26.5175  27.900333  27.909667  27.304889   
           603685  12.606  12.349  12.3380  12.534000  12.560167  12.521556   
           603809  10.810  11.037  11.5795  11.870667  11.197667  10.538333   
           300729  21.792  21.290  20.9780  21.331667  21.864833  22.381111   
           002919  15.726  15.402  15.3895  15.059000  14.925167  14.872444   
           300735  27.568  27.946  28.2110  29.659667  28.550167  28.036444   
           603161  12.594  12.882  12.9340  13.078333  13.136667  13.113333   
           603506  18.504  18.338  18.5880  19.318000  19.841833  20.687444   
           002926  10.378  10.646  11.0360  11.377000  11.038500  10.687000   
           300644  31.998  31.447  32.8565  33.747667  31.833167  30.696000   
           603709  25.778  25.715  25.1540  24.523000  24.164500  24.341444   
           300741  30.550  30.456  29.7095  29.827000  30.071667  29.929778   
           002928  12.520  12.693  12.4410  12.350000  12.105667  11.771778   
           603059  24.708  24.705  25.2310  25.079667  25.587667  25.4198

In [40]:
# ind.query('dfh==True and dfl==True')

,,m5,m10,m20,m30,m60,m90,m120,m250,dfh,dfl
date,code,,,,,,,,,,
2020-04-07,000002,26.6,26.321,27.116,28.316333,29.057667,29.175111,28.573167,28.4054,True,True


In [48]:
drdf=ind.query('dfh==True and dfl==True')

In [49]:
drdf.to_csv('drnhxg.csv')

In [ ]:
x1 = ind.query('res==True').loc[:,['x1','x2','x3','x4']].all(axis=1).groupby(level=1).shift()


In [ ]:
x1[x1==True]

In [ ]:
ind.loc['2008-04-22':'2008-06-19']